# AUTODEALER AI

In [2]:
#import packages
import os
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
#initialization
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [4]:
#System Prompt
system_message = (
    "You are AutoDealer AI for a car dealership. "
    "Answer in one short, courteous sentence. "
    "Be accurate; if unsure, say you don’t know. "
    "If the user asks for price, inventory, test drives, or payments, use tools."
)

In [5]:
# Basic price list keyed by (make, model, year)
vehicle_prices = {
    ("Toyota", "Corolla", 2021): 17950,
    ("Honda", "Civic", 2020): 18500,
    ("Tesla", "Model 3", 2022): 32900,
    ("Ford", "Escape", 2019): 16900,
    ("Hyundai", "Kona", 2021): 19950,
}

In [6]:
# Simple inventory items with features/tags for filtering
inventory = [
    {
        "id": 101, "make": "Toyota", "model": "Corolla", "year": 2021,
        "body_type": "Sedan", "fuel": "Gas", "transmission": "Automatic",
        "mileage": 24000, "price": 17950, "city": "Boston",
        "features": ["adaptive_cruise", "lane_keep", "apple_carplay"],
    },
    {
        "id": 102, "make": "Honda", "model": "Civic", "year": 2020,
        "body_type": "Sedan", "fuel": "Gas", "transmission": "Automatic",
        "mileage": 30000, "price": 18500, "city": "Boston",
        "features": ["sunroof", "heated_seats", "carplay"],
    },
    {
        "id": 103, "make": "Tesla", "model": "Model 3", "year": 2022,
        "body_type": "Sedan", "fuel": "EV", "transmission": "Single-speed",
        "mileage": 12000, "price": 32900, "city": "Cambridge",
        "features": ["autopilot", "fast_charge", "nav"],
    },
    {
        "id": 104, "make": "Ford", "model": "Escape", "year": 2019,
        "body_type": "SUV", "fuel": "Gas", "transmission": "Automatic",
        "mileage": 41000, "price": 16900, "city": "Somerville",
        "features": ["awd", "blind_spot", "power_liftgate"],
    },
    {
        "id": 105, "make": "Hyundai", "model": "Kona", "year": 2021,
        "body_type": "SUV", "fuel": "Gas", "transmission": "Automatic",
        "mileage": 22000, "price": 19950, "city": "Boston",
        "features": ["adaptive_cruise", "lane_keep", "heated_seats"],
    },
]

In [7]:
# Let's generate test-drive slots for the next 5 days, 10am/1pm/3pm
def generate_slots():
    base = datetime.now().replace(minute=0, second=0, microsecond=0)
    cities = ["Boston", "Cambridge", "Somerville"]
    slots = []
    for city in cities:
        for d in range(5):
            for hour in [10, 13, 15]:
                slots.append({
                    "slot_id": f"{city[:3].upper()}-{d}-{hour}",
                    "city": city,
                    "datetime": (base + timedelta(days=d)).replace(hour=hour),
                    "capacity": 2,
                    "booked": 0,
                })
    return slots

test_drive_slots = generate_slots()

In [9]:
#Tool Implementations - Getting Vehicle Price
def get_vehicle_price(make: str, model: str, year: int):
    print(f"Tool get_vehicle_price called for {make} {model} {year}")
    price = vehicle_prices.get((make.title(), model.title(), int(year)))
    return {"found": price is not None, "price": price}

In [10]:
#Search Inventory
def search_inventory(
    make: str | None = None,
    model: str | None = None,
    body_type: str | None = None,
    fuel: str | None = None,
    max_price: float | None = None,
    city: str | None = None,
    features_csv: str | None = None,
):
    print("Tool search_inventory called")
    feat = [f.strip() for f in features_csv.split(",")] if features_csv else []
    results = []
    for item in inventory:
        if make and item["make"].lower() != make.lower(): 
            continue
        if model and item["model"].lower() != model.lower(): 
            continue
        if body_type and item["body_type"].lower() != body_type.lower(): 
            continue
        if fuel and item["fuel"].lower() != fuel.lower(): 
            continue
        if city and item["city"].lower() != city.lower():
            continue
        if max_price is not None and item["price"] > float(max_price):
            continue
        if feat and not all(f in item["features"] for f in feat):
            continue
        results.append(item)
    return {"count": len(results), "items": results[:10]}

In [11]:
#Booking Test Drive
def book_test_drive(city: str, slot_id: str, vehicle_id: int, customer_name: str, customer_email: str):
    print(f"Tool book_test_drive called for {city} slot {slot_id} vehicle {vehicle_id}")
    # Verify vehicle exists in that city
    veh = next((v for v in inventory if v["id"] == int(vehicle_id) and v["city"].lower() == city.lower()), None)
    if not veh:
        return {"success": False, "message": "Vehicle not found in that city."}

    slot = next((s for s in test_drive_slots if s["slot_id"] == slot_id and s["city"].lower() == city.lower()), None)
    if not slot:
        return {"success": False, "message": "Slot not found."}
    if slot["booked"] >= slot["capacity"]:
        return {"success": False, "message": "Slot full."}

    slot["booked"] += 1
    booking_id = f"BK{slot_id}-{vehicle_id}-{slot['booked']}"
    return {
        "success": True,
        "booking_id": booking_id,
        "vehicle": {"id": veh["id"], "title": f"{veh['year']} {veh['make']} {veh['model']}"},
        "slot_time": slot["datetime"].isoformat(),
        "city": slot["city"],
        "customer_name": customer_name,
        "customer_email": customer_email,
    }

In [12]:
#Listing Test Drive Slots
def list_test_drive_slots(city: str):
    print(f"Tool list_test_drive_slots called for {city}")
    slots = [
        {
            "slot_id": s["slot_id"],
            "datetime": s["datetime"].isoformat(),
            "available": max(0, s["capacity"] - s["booked"]),
        }
        for s in test_drive_slots
        if s["city"].lower() == city.lower()
    ]
    return {"count": len(slots), "slots": slots[:15]}

In [13]:
#Estimate Payment
def estimate_payment(price: float, down_payment: float, apr_percent: float, term_months: int):
    """
    Simple fixed-rate loan monthly payment (P * r * (1+r)^n) / ((1+r)^n - 1)
    where P = price - down, r = APR/12, n = term_months
    """
    print("Tool estimate_payment called")
    principal = max(0.0, float(price) - float(down_payment))
    r = float(apr_percent) / 100.0 / 12.0
    n = int(term_months)
    if principal == 0 or n <= 0:
        return {"monthly": 0.0, "total_interest": 0.0}
    if r == 0:
        monthly = principal / n
        total_interest = 0.0
    else:
        monthly = principal * r * (1 + r) ** n / ((1 + r) ** n - 1)
        total_interest = monthly * n - principal
    return {"monthly": round(monthly, 2), "total_interest": round(total_interest, 2)}

In [14]:
#Tool Schema - For function calling
price_function = {
    "name": "get_vehicle_price",
    "description": "Get MSRP/lot price for a specific car.",
    "parameters": {
        "type": "object",
        "properties": {
            "make": {"type": "string", "description": "Car make, e.g., Toyota"},
            "model": {"type": "string", "description": "Car model, e.g., Corolla"},
            "year": {"type": "integer", "description": "Car model year, e.g., 2021"},
        },
        "required": ["make", "model", "year"],
        "additionalProperties": False,
    },
}

In [15]:
#Inventory Function
inventory_function = {
    "name": "search_inventory",
    "description": "Search available vehicles by filters.",
    "parameters": {
        "type": "object",
        "properties": {
            "make": {"type": "string"},
            "model": {"type": "string"},
            "body_type": {"type": "string", "description": "Sedan, SUV, etc."},
            "fuel": {"type": "string", "description": "Gas, Hybrid, EV"},
            "max_price": {"type": "number"},
            "city": {"type": "string"},
            "features_csv": {
                "type": "string",
                "description": "Comma-separated features like 'awd, heated_seats'",
            },
        },
        "required": [],
        "additionalProperties": False,
    },
}

In [16]:
#Slot function
slots_function = {
    "name": "list_test_drive_slots",
    "description": "List available test-drive slots in a city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {"type": "string", "description": "City of the dealership"},
        },
        "required": ["city"],
        "additionalProperties": False,
    },
}

In [17]:
#Booking function
booking_function = {
    "name": "book_test_drive",
    "description": "Book a test drive for a vehicle in a given city and slot.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {"type": "string"},
            "slot_id": {"type": "string"},
            "vehicle_id": {"type": "integer"},
            "customer_name": {"type": "string"},
            "customer_email": {"type": "string"},
        },
        "required": ["city", "slot_id", "vehicle_id", "customer_name", "customer_email"],
        "additionalProperties": False,
    },
}

In [18]:
#Payment function
payment_function = {
    "name": "estimate_payment",
    "description": "Estimate monthly payment for a car loan.",
    "parameters": {
        "type": "object",
        "properties": {
            "price": {"type": "number"},
            "down_payment": {"type": "number"},
            "apr_percent": {"type": "number"},
            "term_months": {"type": "integer"},
        },
        "required": ["price", "down_payment", "apr_percent", "term_months"],
        "additionalProperties": False,
    },
}

In [19]:
#All the tools in one single list
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": inventory_function},
    {"type": "function", "function": slots_function},
    {"type": "function", "function": booking_function},
    {"type": "function", "function": payment_function},
]

In [20]:
#Tool Call Handler (routes by function name)

def handle_tool_call(message):
    call = message.tool_calls[0]
    name = call.function.name
    args = json.loads(call.function.arguments or "{}")

    if name == "get_vehicle_price":
        result = get_vehicle_price(**args)
    elif name == "search_inventory":
        result = search_inventory(**args)
    elif name == "list_test_drive_slots":
        result = list_test_drive_slots(**args)
    elif name == "book_test_drive":
        result = book_test_drive(**args)
    elif name == "estimate_payment":
        result = estimate_payment(**args)
    else:
        result = {"error": f"Unknown tool {name}"}

    response = {
        "role": "tool",
        "content": json.dumps(result),
        "tool_call_id": call.id,
    }
    return response

In [21]:
#Chat Function

def chat(message, history):
    # Gradio `type="messages"` passes history as list of dicts with role/content
    messages = [{"role": "system", "content": system_message}] + history + [
        {"role": "user", "content": message}
    ]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    # Single-step function call (mirrors your example)
    if response.choices[0].finish_reason == "tool_calls":
        tool_message = response.choices[0].message
        tool_response = handle_tool_call(tool_message)
        messages.append(tool_message)
        messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [22]:
#Launching UI
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Tool get_vehicle_price called for Toyota Corolla 2021
Tool get_vehicle_price called for Toyota Corolla 2021
